### Importing Necessary Libraries

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

### Loading and Splitting the dataset into train and test

In [6]:
data = load_breast_cancer()
X = data.data
y = data.target
df = pd.DataFrame(data=data.data, columns=data.feature_names)
column_names = list(df.columns.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=14)

### Definition of ACO function

In [7]:
def ACO(X_train, X_test, y_train, y_test, ants, iterations):
    features = X_train.shape[1]
    # Initialize the pheromone matrix
    pheromone = np.ones(features)
    # Define the parameters of the algorithm
    alpha = 1.0 # Pheromone importance
    beta = 2.0 # Heuristic importance
    rho = 0.5 # Pheromone evaporation rate
    # Define the fitness function
    def fitness(X_train, X_test, y_train, y_test, mask):
        clf = KNeighborsClassifier(n_neighbors=3)
        clf.fit(X_train[:, np.ravel(mask)], y_train)
        return clf.score(X_test[:, np.ravel(mask)], y_test)
    # Iterate through the algorithm
    for i in range(iterations):
        # Generate the ant solutions
        solutions = []
        for j in range(ants):
            mask = np.full(features, False, dtype=bool)
            for k in range(features):
                # Calculate the probabilities for each feature
                prob = pheromone[k]**alpha * (1.0 / (1.0 + np.exp(-beta * X_train[:,k])))
                # Select the feature based on the probabilities
                mask[k] = np.random.choice(a=[True, False], p = [prob[k],1-prob[k]])
            solutions.append(mask)
        # Update the pheromone matrix
        for j in range(ants):
            mask = solutions[j]
            fitness_j = fitness(X_train, X_test, y_train, y_test, mask)
            for k in range(features):
                if mask[k]:
                    pheromone[k] = (1.0 - rho) * pheromone[k] + rho * fitness_j
                else:
                    pheromone[k] = (1.0 - rho) * pheromone[k]
    best_mask = solutions[0]
    best_fitness = fitness(X_train, X_test, y_train, y_test, best_mask)
    for mask in solutions[1:]:
        fitness_i = fitness(X_train, X_test, y_train, y_test, mask)
        if fitness_i > best_fitness:
            best_mask = mask
            best_fitness = fitness_i
    return best_mask, best_fitness

### Test the ACO function and print results

In [12]:
ants = 10
iterations = 10
best_mask, best_fitness = ACO(X_train, X_test, y_train, y_test, ants, iterations)
selected_features=[]
for i in range(len(best_mask)):
    if(best_mask[i]==True):
        selected_features.append(column_names[i])
print("All features:", column_names)
print("Selected features:", selected_features)
print("Accuracy:", best_fitness)

All features: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
Selected features: ['mean radius', 'texture error', 'worst radius']
Accuracy: 0.935672514619883
